In [197]:
import sqlite3
from sqlite3 import Error
 
 
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
        
    return None

In [198]:
def create_database(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [199]:
database_name = "allergy_forecast"
def add_record(conn, record):
    sql = "INSERT INTO " + database_name + "(region, name, month, decade, intensity) VALUES(?,?,?,?,?)"
    cur = conn.cursor()
    cur.execute(sql, record)
    return cur.lastrowid

In [200]:
import requests
from bs4 import BeautifulSoup

prefix_url = 'http://www.alergen.info.pl/kalendarz_pylenia_roslin.php?r=region_'

def get_forecast(region, conn):
    r = requests.get(url + str(region))
    soup = BeautifulSoup(r.text, 'html.parser')
    
    forecast = soup.find_all('tr')
    for i in range(2, len(results)): #2nd is first meaningful tag
        allergen_forecast = forecast[i].find_all('td')
        name =  allergen_forecast[0].text
        
        '''It is hard coded because word DĄB is converted through BeautifulSoup
        to some another weird word which consists of invisible characters '''
        if(name[0] == 'D'):
            name = 'DAB'        

        for j in range(2, len(allergen_forecast)):
            attr = allergen_forecast[j]['class'][1]
            if(attr == "p_yellow"):
                intensity = 1
            elif(attr == "p_orange"):
                intensity = 2
            elif(attr == "p_red"):
                intensity = 3
            else:
                intensity = 0
            month = ((int) ((j + 1) / 3))
            decade = (j + 1) % 3 + 1
            record = (region, name, month, decade, intensity);
            add_record(conn, record)

In [201]:
def main():
    database = "allergy_forecast.db"
 
    sql_create_forecast_table = """ CREATE TABLE IF NOT EXISTS allergy_forecast (
                                        _id integer PRIMARY KEY AUTOINCREMENT,
                                        region integer,
                                        name text NOT NULL,
                                        month integer,
                                        decade integer,
                                        intensity integer,
                                        UNIQUE(region, name, month, decade)
                                    ); """
 
    # create a database connection
    conn = create_connection(database)
    if conn is not None:
        create_database(conn, sql_create_forecast_table)
    else:
        print("Error! cannot create the database connection.")
        
    with conn:
        url = 'http://www.alergen.info.pl/kalendarz_pylenia_roslin.php?r=region_'
        for i in range(1, 5):
            get_forecast(i, conn)

In [202]:
if __name__ == '__main__':
    main()